# Game Schedule for U.S. National Football League

Fall 2020 Optimization 

Jianing Zhang, Junyi Qian, Yue Zheng, Ziqiao Yan


NFL is the professional American football league consisting of 32 teams, divided equally between the National Football Conference (NFC) and the American Football Conference (AFC).
Each conference is further divided into four division of four clubs and every NFL team is based in the contiguous United States. Since the analysis including all the teams right-away was estimated to be computationally intensive, Liang was asked to develop a preliminary analysis for a subset of teams, namely for teams in the East, North, and South divisions. This setup was agreed by the NFL schedulers to approximate the overall problem sufficiently well to come up with recommendations. Figure 1 shows all the 24 teams that are included in the analysis.

The scheduling of games for the selected subset of teams is constrained by the following four main rules:

1. The season was limited to 12 weeks.
2. Each team would play once per week.
3. All 12 games that a team played would need to be against a different opponent.
4. Each team would play at most six home games (i.e., on their home stadium).

Whereas the overall goal was to reduce the total distance that the teams travelled, there were also other considerations, such as the fairness of the game schedule for all teams and the fairness of the game schedule for broad casting companies. With these thoughts in mind, Liang embarked on the analysis of the NFL game scheduling.

## Assignment


In this workshop, you will build an integer programming model and use the results of the optimization to support your recommendations. Make a report of about 3 pages (excluding figures, graphs and appendices) summarizing your results, your interpretation of the results and your conclusions and recommendations. You can include graphs and outputs in an appendix, but make sure to summarize and interpret the results in your report as well.

In [3]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

In [11]:
#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}

In [12]:
W=set(list(range(0,12))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
 #set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)

In [ ]:
########################################
########### MODEL ######################
########################################

# Create an empty model
m1= gp.Model(name="NFL1")

## 1. The Plain Vanilla Schedule

To start, develop a schedule that <strong>minimizes the total travel distance of all teams</strong>. The distances between the home stadiums of each team are shown in miles in the file “distance.cvs”. When calculating the distances that teams travel, you can assume that after each game the team that played away will travel back to their home stadium before the next game. Include in your model the four main rules.

#### - What are the decision variables in this problem?

The decision variable should be considered as a binary indication of weather home team $h$ would play against away team $a$ on week $w$. For example, if $x_{h,a,w}$ equals to 1, is means that the home team $h$ would play against with away team $a$ on week $w$. Otherwise, the game does not happen. 

#### - What is the objective function? What are the constraints?

The objective function which minimizes the total travel distance of all teams should be the summed product of all the teams travel distances through out the 12 weeks. If we assume the travel distance from the origin to destination as $d_{h,a}$， the objective function should be defined as

$min\sum_{h,a,w}x_{h,a,w}\cdot 2d_{h,a} $

In [ ]:
# ADD DECISION VARIABLES HERE
home=len(E.columns)
away=len(E.columns)
#teams=len(E.columns)
weeks=12
x=m1.addVars(home, away, weeks, vtype=GRB.BINARY, name="x")

In [ ]:
########################################
#### CONSTRAINTS & OBJ FUNCTIONS #######
########################################

# ADD OBJECTIVE FUNCTION HERE
obj = sum(sum(2*E.iloc[h,a] * x[h,a,w] for h in range(home) for a in range(away)) for w in range(weeks))
m1.setObjective(obj, GRB.MINIMIZE)

The constraints for the schedualing, as discussed in the problem statement as follows

1. The season was limited to 12 weeks.
2. Each team would play once per week.
3. All 12 games that a team played would need to be against a different opponent.
4. Each team would play at most six home games (i.e., on their home stadium).

#### - Write down the mathematical formulation of the problem.

$min\sum_{h,a,w}x_{h,a,w}\cdot 2d_{h,a} $

subject to 

1. The season was limited to 12 weeks:

$\sum_{h}\sum_{w}[x_{h,a,w}+x_{h,a,w}]=12$

In [ ]:
con1=m1.addConstrs(sum(x[h,a,w] + x[h,a,w] for h in range(home) for w in range(weeks)) == 12 
               for a in range(away))

2. Each team would play once per week.

$\sum_{h}[x_{h,a,w}+x_{a,h,w}]=1$

In [ ]:
con2=m1.addConstrs(sum(x[h,a,w] +x[a,h,w] for h in range(home)) == 1 
              for w in range(weeks) for a in range(away))

3. All 12 games that a team played would need to be against a different opponent.

$\sum_{w}[x_{h,a,w}+x_{a,h,w}]<=1$

In [ ]:
con3=m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for w in range(weeks)) <= 1 
                       for h in range(home) for a in range(away))

4. Each team would play at most six home games (i.e., on their home stadium).

$\sum_{h}\sum_{w}x_{h,a,w}<=6$

In [ ]:
con4=m1.addConstrs(sum(x[h,a,w] for h in range(home) for w in range(weeks) ) <= 6 
                   for a in range(away))

Update and optimize the model

In [ ]:
m1.update()
m1

In [ ]:
m1.optimize()

#### Develop your optimization model and save it as “nfl 1.py”. 

(i) the minimum total distance travelled by all teams 

(ii) illustrate the optimal schedule for Cleveland Browns. How does the home and away game pattern look like for the Cleveland Browns?


In [ ]:
########################################
########### PRINT RESULTS ##############
########################################

# ADD PRINTING HERE:



# print the total distance travelled by all teams

if m1.status == GRB.OPTIMAL:
    # Print solution
    print("\nTotal distance: {}\n". format(m1.ObjVal))    

# print the optimal schedule for the decision variables

print('\nDecision variables:')
for v in m1.getVars():
    if v.x== 1:
        s=("%s : %g" % (v.varName, v.obj))
        print(s)

## 2. Game Pattern and Bye Week Constraints


The NFL schedulers asked whether it would be possible to adjust the plain vanilla schedule (which followed the four main rules) to obtain a schedule in which no team would play more than two consecutive games at home or two consecutive games away. The logic behind these game pattern constraints was to provide a fairer schedule for the teams in terms of them having similar home/away game patterns.

#### -What is the mathematical formulation for the game pattern constraints?

In order to have no team play no more than two consecutive games at home or away, we need to have another two binary decision variables $y_{h,w}$ and $z_{h,w}$. 

$y_{h,w}$ denotes to weather home team $h$ plays at home in weeks $w$ and $w+1$. If $y_{h,w}=1$, it means that home team $h$ plays at home on weeks $w$ and $w+1$, $y_{h,w}=0$ otherwise. 

$z_{h,w}$ denotes to weather home team $h$ plays away in weeks $w$ and $w+1$. If $z_{h,w}=1$, it means that home team $h$ plays away on weeks $w$ and $w+1$, $z_{h,w}=0$ otherwise. 

Adjust the model developed in part 1. by incorporating the game pattern constraints and save the model as “nfl 2a.py”. 

#### -What are the implications of the game pattern constraints? 

If $y_{h,w}$ and $z_{h,w}$ equals to 1, it means the home team $h$ played two consecutive games at home or away. 


5. No team play no more than two consecutive games at home. 

$\sum_{w}y_{h,w}<=1$

$\sum_{a}[x_{h,a,w}+x_{h,a,w+1}]<=1+y_{h,w}$

In [ ]:
con5a=m1.addConstrs(sum(y[h,w] for w in range(weeks)) <= 1 
                   for h in range(home))

In [ ]:
con5b=m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] for a in range(away)) <= 1 + y[h,w] 
               for h in range(home) for w in range(weeks-1))

6. No team play no more than two consecutive games away.

$\sum_{w}z_{h,w}<=1$

$\sum_{a}[x_{h,a,w}+x_{h,a,w+1}]<=1+z_{h,w}$

In [ ]:
con6a=m1.addConstrs(sum(z[h,w] for w in range(weeks)) <= 1 
                   for h in range(home))

In [ ]:
con6b=m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] for a in range(away)) <= 1 + z[h,w] 
               for h in range(home) for w in range(weeks-1))

- What happened to the minimum total distance travelled by all teams and the optimal schedule for Cleveland Browns?

To further reduce the fatigue caused by the competition and travelling, the NFL schedulers asked to incorporate in the schedule a week when there are no games. This one-week long break is also referred as the bye week. Suppose that the schedulers requested the bye week to be assigned on week 9, which means that there are no games at week 9, and that the whole season is extended by a week up to 13 weeks.


#### -What are the required changes for the mathematical formulation to incorporate also the bye week constraint?

Adjust the model “nfl 2a.py” by incorporating the bye week constraint and save the model as “nfl 2b.py”. 

#### -What are the consequences for the minimum total distance travelled by all teams and the optimal schedule for Cleveland Browns?

#### -Would you recommend adding the game pattern and bye week constraints?

In [ ]:
哔哔哔哔哔哔不不